# Исследование надёжности заёмщиков

В датасете представлены данные о заёмщиках банка.

Цель исследования - понять, какие параметры влияют на возврат кредита в срок.

Данные в формате `.csv`.

О качестве данных информации нет, поэтому необходимо провести предобработку.

Проект будет выполнен в несколько этапов:
1. [Открытие данных](#start)
2. [Предобработка данных](#preprocessing)
   * [Обработка пропусков](#na)
   * [Замена типа данных](#type_change)
   * [Обработка дупликатов](#duplicated)
   * [Лемматизация](#lemmatize)
   * [Категоризация данных](#category)

3. [Исследование заёмщиков](#research)
4. [Общий вывод](#conclusion)
5. [Чек-лист](#check_list)

<a id='start'></a>
## Открытие данных

In [1]:
import pandas as pd
from pymystem3 import Mystem # импорт класса Mystem

Ознакомимся с общей информацией о таблице.

In [2]:
data = pd.read_csv('data.csv')
#  посмотрим первые 5 строк таблицы
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

В каждой строке таблицы представлены данные о заёмщиках. Все названия столбцов корректно оформлены - в змеином регистре. В столбцах `'days_employed'` и `'total_income'` количество строк не совпадает с количеством строк датафрейма, что говорит о том, что в данных имеются пропуски. 
Формат значений в этих столбцах - float64, а также описание этих столбцов (в первом случае - стаж, во втором - уровень ежемесячного дохода) говорят нам о том, что в данном случае мы имеем дело с количественными значениями.

<a id="preprocessing"></a>
##  Предобработка данных 

<a id="na"></a>
### Обработка пропусков

Посчитаем количество пропущенных значений в целом по датафрейму.

In [3]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Как видно, количество пропущенных значений в столбцах со стажем и уровнем дохода совпадает.
Количество пропущенных значений составляет 2174, что больше 10% данных. Если их просто удалить, результаты могут сильно пострадать.

Создадим таблицу, в которой будут только те строки, в которых есть пропуски по столбцу с 'total_income':

In [4]:
# с помощью фильтрации создадим таблицу
data_nan = data[data['total_income'].isna()]
data_nan.head() # выведем на экран первые 5 строк

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


Теперь посчитаем в этой таблице количество пропусков по столбцу days_employed, если их количество равно 2174, то можно сделать вывод, что строки с пропусками у столбцов с пропусками совпадают:

In [5]:
# с помощью фильтрации найдём строки, в которых есть пропуски в столбце 'days_employed' 
# и с помощью len найдём количество строк.
len(data_nan[data_nan['days_employed'].isna()])

2174

* Все строки в столбце со стажем с пропусками, можно делать вывод, что пропущенные значения в столбцах со стажем и общим доходом совпадают.
* Учитывая, что пропуски в одинаковых строках и в столбцах, природа их возникновения, вероятно, одинаковая - скорее всего банк не смог получить по данным клиентам информацию от работадателя. 
* Пропущенные данные являются количественными.
* В данном исследовании пропуски в столбце со стажем не являются критичными, они никак не будут влиять на результаты.
* Пропуски же в столбце с доходами не дадут провести исследование.

***Устранение пропущенных значений в столбце с ежемесячным уровнем доходов***

Количественные значения можно заменить средним значением или медианой. В случае оценки зарплат, лучше всего подходит медиана, так как выбросы в виде слишком высоких или слишком низких зарплат могут внести весомый вклад в оценку.

Также, есть предположение, что медианная зарплата для разных значений income_type будет сильно отличаться. К примеру, можно предположить, что медианная зарплата пенсионера явно меньше, чем медиана компаньона, поэтому было бы логичнее подставлять пенсионерам медиану пенсионеров и тд. 
Посмотрим, каким позициям в income_type соответствуют пропуски и какое количество:

In [6]:
# получим наименования типов занятости и количество пропусков в каждом из них
data[data['total_income'].isna()]['income_type'].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

Посчитаем соответствующие медианы:

In [7]:
# применяем логическую фильтрацию и посчитаем медиану с помощью median() для каждого типа занятости, а также в целом
employee_median_income = data[data['income_type'] == 'сотрудник']['total_income'].median()
companion_median_income = data[data['income_type'] == 'компаньон']['total_income'].median()
pensioner_median_income = data[data['income_type'] == 'пенсионер']['total_income'].median()
gov_median_income = data[data['income_type'] == 'госслужащий']['total_income'].median()
businessman_median_income = data[data['income_type'] == 'предприниматель']['total_income'].median()
total_median_income = data['total_income'].median()
# Выведем на экран значения медиан, для сравнения
print('Медиана сотрудников:', employee_median_income)
print('Медиана компаньонов:', companion_median_income)
print('Медиана пенсионеров:', pensioner_median_income)
print('Медиана госслужащих: ', gov_median_income)
print('Медиана предпринимателей:', businessman_median_income)
print('Медиана общая:', total_median_income)

Медиана сотрудников: 142594.39684740017
Медиана компаньонов: 172357.95096577113
Медиана пенсионеров: 118514.48641164352
Медиана госслужащих:  150447.9352830068
Медиана предпринимателей: 499163.1449470857
Медиана общая: 145017.93753253992


Как видно, медиана пенсионеров, компаньонов и предпринимателей довольно сильно отличаются друг от друга и от общей, поэтому лучше заменить пропущенные значения согласно типу занятости.

In [8]:
# при помощи логической индексации заполняем пропуски в строках согласно значению типа занятости
# для этого обращаемся с помощью loc. к строкам с соответствующим 'income_type', и столбцам 'total_income'
# присваем значение медиан в этих строках в столбце 'total_income' 
data.loc[data['income_type'] == 'пенсионер', 'total_income'] = data['total_income'].fillna(value=pensioner_median_income)
data.loc[data['income_type'] == 'сотрудник', 'total_income'] = data['total_income'].fillna(value=employee_median_income)
data.loc[data['income_type'] == 'компаньон', 'total_income'] = data['total_income'].fillna(value=companion_median_income)
data.loc[data['income_type'] == 'предприниматель', 'total_income'] = data['total_income'].fillna(value=businessman_median_income)
data.loc[data['income_type'] == 'госслужащий', 'total_income'] = data['total_income'].fillna(value=gov_median_income)

Посчитаем количество пропусков:

In [9]:
# Проверим теперь количество пропусков в столбце 'total_income'
data['total_income'].isna().sum()

0

Пропуски в столбце со значениями уровня дохода устранены.

***Устранение пропущенных значений в столбце со стажем***

Стаж вероятнее всего сильно зависит от возраста. Но, так как в данном исследовании влияние стажа никак не будет учитываться, замением его для простоты на значение медианы для всех строк.

In [10]:
# посчитаем значением медианы стажа
days_employed_median = data['days_employed'].median() 
print('Значение медианы для столбца со стажем:', days_employed_median)

Значение медианы для столбца со стажем: -1203.369528770489


Значение медианы получилось отрицательным, и действительно, в таблице видно, что некоторые значения стажа отрицательные.
Скорее всего это связано с тем, как заполняли поле со стажем.
Скорее всего минус - это тире. Соответственно, можно все отрицательные значения стажа заменить на положительные.

In [11]:
# выведем количество отрицательных значений в столбце 'days_employed'
data[data['days_employed'] < 0]['days_employed'].count()

15906

Как видно, данная ошибка встречается в большинстве случаев. Неплохо бы проверить, почему так получается... Ошибка явно системная.
Заменим отрицательные значения на положительные:

In [12]:
# применим функцию abs() к столбцу 'data_employed', чтобы заменить отрицательные значения на положительные
# выведем количество отрицательных значений в столбце 'days_employed'
data['days_employed'] = data['days_employed'].apply(abs)
data[data['days_employed'] < 0]['days_employed'].count()

0

In [13]:
# посчитаем значением медианы стажа
days_employed_median = data['days_employed'].median() 
print('Значение медианы для столбца со стажем:', days_employed_median)

Значение медианы для столбца со стажем: 2194.220566878695


Заменим пропущенные значения на медиану:

In [14]:
# заменим пропущенные значения на медиану
data['days_employed'] = data['days_employed'].fillna(value=days_employed_median)

In [15]:
# посчитаем теперь количество пропусков в таблице в целом
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Вывод**
* Пропущенные значения встречаются в одинаковых строках, вероятно не удалось получить данные из какого-то источника;
* пропущенные данные являются количественными;
* пропуски в столбце с ежемесячным доходом заменены на медианы соответствующих типов занятости;
* значения в столбце со стажем содержали артефакты - отрицательные значения. Предположительно из-за некорректного заполнения;
* количество отрицательных значений больше половины, что говорит о системности данной проблемы;
* количество отрицательных значений больше половины, что говорит о системности данной проблемы;
* отрицательные значения были заменены на положительные;
* пропущенные значения были заменены на медиану по данным без разделения данных на группы по возрасту.

<a id="type_change"></a>
### Замена типа данных

Выведем общую информацию о таблице:

In [16]:
data.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 11.3 MB


Количество памяти занимаемой таблицей - 11,3 Мб.
Тип данных в столбцах со стажем и ежемесячным доходом - вещественные. В остальных ячейках int64, что для большинства случае избыточно. Заменим их на целочисленные. Существует несколько типов int:
* int8 - максимальное число - 127, минимальное - -128 размер занимаемой памяти 1 байт
* int16 - максимальное число - 32767,минимальное - -32768 размер занимаемой памяти 2 байт
* int32 - максимальное число - 2147483647,минимальное - -2147483648 размер занимаемой памяти 4 байт
* int64 - максимальное число - 9223372036854775807,минимальное - -9223372036854775808 размер занимаемой памяти 8 байт
Узнаем максимальные значения, которые принимают значения в столбцах с float64 и int64, чтобы определить, какой тип данных выбрать, чтобы при этом выделить как можно меньше памяти: 

In [17]:
for column in data.columns:
    if (data[column].dtype == 'int64') or (data[column].dtype == 'float64'):
        print(f'Макисмальное значение в столбце {column} = {data[column].max()}')
        print(f'Минимальное значение в столбце {column} = {data[column].min()}')
    

Макисмальное значение в столбце children = 20
Минимальное значение в столбце children = -1
Макисмальное значение в столбце days_employed = 401755.40047533
Минимальное значение в столбце days_employed = 24.14163324048118
Макисмальное значение в столбце dob_years = 75
Минимальное значение в столбце dob_years = 0
Макисмальное значение в столбце education_id = 4
Минимальное значение в столбце education_id = 0
Макисмальное значение в столбце family_status_id = 4
Минимальное значение в столбце family_status_id = 0
Макисмальное значение в столбце debt = 1
Минимальное значение в столбце debt = 0
Макисмальное значение в столбце total_income = 2265604.028722744
Минимальное значение в столбце total_income = 20667.26379327158


* максимальное значение столбца children=20, подойдёт int8;
* максимальное значение столбца days_employed=401755, подойдёт int32, но стоит заметить, что данное значение в днях соотвествует более тысячи лет, ниже проверим, что с этим можно сделать;
* максимальное значение столбца dob_years=75, подойдёт int8;
* максимальное значение столбца education_id=4, подойдёт int8;
* максимальное значение столбца family_status_id=4, подойдёт int8;
* максимальное значение столбца debt=1, подойдёт int8;
* максимальное значение столбца total_income=2265604, подойдёт int32;

Также, для всех столбов можно брать uint, так как все числа должны быть положительными, но, учитывая, что встречаются отрицательные значения там, где их быть не должно, более стабильно в данном случае использовать просто int.

Заменим типы данных:

In [18]:
data['children'] = data['children'].astype('int8')
data['days_employed'] = data['days_employed'].astype('int32')
data['dob_years'] = data['dob_years'].astype('int8')
data['education_id'] = data['education_id'].astype('int8')
data['family_status_id'] = data['family_status_id'].astype('int8')
data['debt'] = data['debt'].astype('int8')
data['total_income'] = data['total_income'].astype('int32')
data.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int8
days_employed       21525 non-null int32
dob_years           21525 non-null int8
education           21525 non-null object
education_id        21525 non-null int8
family_status       21525 non-null object
family_status_id    21525 non-null int8
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int8
total_income        21525 non-null int32
purpose             21525 non-null object
dtypes: int32(2), int8(5), object(5)
memory usage: 10.4 MB


Проверим столбцец со стажем, посчитаем количество людей со стажем более 100 000 дней.

In [19]:
data[data['days_employed'] > 100000]['days_employed'].count()

3445

Проверим столбцец со стажем, посчитаем количество пропусков со стажем более 20 000, что соответствует стажу в 55 лет.

In [20]:
data[data['days_employed'] > 20000]['days_employed'].count()

3445

Получилось такое же количество. То есть значения с ошибкой колеблются в диапозоне 100-400 тысяч - они одного порядка. Посмотрим мединанный стаж:

In [21]:
data['days_employed'].median()

2194.0

Порядок медианы - тысячи, порядок явно ошибочных значений - сотни тысяч. Ошибка в 2 порядка, исправим ее, поделив все значения выше 100 тысяч на 100.

In [22]:
data.loc[data['days_employed'] > 100000, 'days_employed'] = data['days_employed'] / 100
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.00,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.00,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.00,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.00,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,3402.66,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


Приведём теперь столбец к формату int16, так как максимальное значение - менее 20 000.

In [23]:
data['days_employed'] = data['days_employed'].astype('int16')
data.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int8
days_employed       21525 non-null int16
dob_years           21525 non-null int8
education           21525 non-null object
education_id        21525 non-null int8
family_status       21525 non-null object
family_status_id    21525 non-null int8
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int8
total_income        21525 non-null int32
purpose             21525 non-null object
dtypes: int16(1), int32(1), int8(5), object(5)
memory usage: 10.4 MB


Количество занимаемой памяти теперь 10,4 Мб.

In [24]:
print(f'Количество занимаемой памяти уменьшилось на {100-(10.4/11.3)*100}%')

Количество занимаемой памяти уменьшилось на 7.964601769911511%


Заменим тип данных в столбцах `'family_status'` и `'education' `на `'category'`. `Category`  - занимает в каждой ячейке 1 байт, как int8.

In [25]:
data['family_status'] = data['family_status'].astype('category')
data['education'] = data['education'].astype('category')
data.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int8
days_employed       21525 non-null int16
dob_years           21525 non-null int8
education           21525 non-null category
education_id        21525 non-null int8
family_status       21525 non-null category
family_status_id    21525 non-null int8
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int8
total_income        21525 non-null int32
purpose             21525 non-null object
dtypes: category(2), int16(1), int32(1), int8(5), object(3)
memory usage: 6.1 MB


Данная замена позволила уменьшить вес таблицы с 10,4 MB до 6,1 MB.

In [48]:
print(f'Количество занимаемой памяти уменьшилось на {100-(6.1/11.3)*100}%')

Количество занимаемой памяти уменьшилось на 46.01769911504425%


**Вывод**

Типы данных в предлагаемой таблице заменены на более оптимальные с точки зрения занимаемой памяти. Таким образом, количество занимаемой памяти удалось уменьшить на 46%

<a id="duplicated"></a>
### Обработка дубликатов

Обработаем явные дупликаты:

In [26]:
# посчитаем явные дупликаты
data.duplicated().sum()

54

In [27]:
# удалим явные дупликаты
data = data.drop_duplicates().reset_index(drop=True)
# посчитаем явные дупликаты
data.duplicated().sum()

0

Обработаем неявные дупликаты:

In [28]:
# посчитаем количество уникальных значений в столбце 'education' с помощью метода value_counts()
data['education'].value_counts()

среднее                13705
высшее                  4710
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   273
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

В данном столбце много дубликатов связанных с регистром букв - строчные и заглавные. Уберём их, применив метод str.lower()

In [29]:
# приведем все значение в столбце 'education' к единому регистру с помощью str.lower()
data['education'] = data['education'].str.lower()
# применим метод value_counts(), чтобы посчитать обновлённые количества значений
data['education'].value_counts()

среднее                15188
высшее                  5251
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Проверим наличие неявных дупликатов в столбце с семейным положением:

In [30]:
# посчитаем количество уникальных значений в столбце 'family_status' с помощью метода value_counts()
data['family_status'].value_counts()

женат / замужем          12344
гражданский брак          4163
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

В столбце с семейным положением дупликатов связанных с типом регистра не обнаружено. Проверим остальные:

Проверим наличие неявных дупликатов в столбце с типом дохода:

In [31]:
# посчитаем количество уникальных значений в столбце 'income_type' с помощью метода value_counts()
data['income_type'].value_counts()

сотрудник          11091
компаньон           5080
пенсионер           3837
госслужащий         1457
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64

Проверим наличие неявных дупликатов в столбце с количеством детей:

In [32]:
# посчитаем количество уникальных значений в столбце 'children с помощью метода value_counts()
data['children'].value_counts()

 0     14107
 1      4809
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

В столбце с количеством детей есть дубликат связанный с отрицательным значением. А также, можно заметить, что довольно много людей с количеством детей - 20, что выглядит неправдоподобно. Скорее всего это дупликат, заменим его на количество детей = 2. 

In [33]:
# применим функцию abs методом apply ко всему столбцу()
data['children'] = data['children'].apply(abs)
# # посчитаем количество уникальных значений в столбце 'children с помощью метода value_counts()
data['children'].value_counts()

0     14107
1      4856
2      2052
3       330
20       76
4        41
5         9
Name: children, dtype: int64

In [34]:
# точечно заменим значения количества детей 20 на 2 с помощью логической индексации
data.loc[data['children'] == 20, 'children'] = 2
data['children'] = data['children'].astype('int8')
# # посчитаем количество уникальных значений в столбце 'children с помощью метода value_counts()
data['children'].value_counts()

0    14107
1     4856
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

**Вывод**

* Были посчитаны явные дупликаты и удалены методом `drop_duplicates()`
* были выявлены неявные дупликаты в столбце 'education' связанные с регистром. Для посчёта и выявления был использован метод `value_counts()`;
* с помощью метода `str.lower()` все значения в столбце `'education'` были приведены к нижнему регистру;
* в других столбцах дупликаты не были выявлены
* возможная причина появления дупликатов с регистром является то, что не было явных правил по заполнению, допускалось заполнение и верхним и нижним регистром;
* в столбце с количеством детей были неявные дупликаты в виде количества детей равного отрицательному значению и значению 20 вместо 2. Возможно, 20 это семьи, количество детей в которых больше 5, в любом случае, 76 - это меньше одного процента данных.


<a id="lemmatize"></a>
### Лемматизация

Лемматизация - приведение слова к его словарной форме (лемме). 

Проведем лемматизацию столбца с целями кредита: 

In [35]:
m = Mystem() # создаем объект класса Mystem
# создаём новый столбец с леммами целей кредита, для этого применяем метод класса Mystem - lemmatize к столбцу 'purpose'
data['purpose_lemmas'] = data['purpose'].apply(m.lemmatize) # создаём новый столбец с леммами целей кредита
data['purpose_lemmas'].value_counts() # количество уникальных списков с леммами
# мой ноутбук пытался час сделать эту операцию, после чего я перестал делать проект локально и перенес всё в тренажер
# здесь - всего пара секунд

[автомобиль, \n]                                          972
[свадьба, \n]                                             793
[на,  , проведение,  , свадьба, \n]                       773
[сыграть,  , свадьба, \n]                                 769
[операция,  , с,  , недвижимость, \n]                     675
[покупка,  , коммерческий,  , недвижимость, \n]           662
[операция,  , с,  , жилье, \n]                            652
[покупка,  , жилье,  , для,  , сдача, \n]                 652
[операция,  , с,  , коммерческий,  , недвижимость, \n]    650
[покупка,  , жилье, \n]                                   646
[жилье, \n]                                               646
[покупка,  , жилье,  , для,  , семья, \n]                 638
[строительство,  , собственный,  , недвижимость, \n]      635
[недвижимость, \n]                                        633
[операция,  , со,  , свой,  , недвижимость, \n]           627
[строительство,  , жилой,  , недвижимость, \n]            625
[покупка

**Вывод**

Таким образом, с помощью лемматизации удалось выделить леммы (словарные формы) слов в запросах. Словарные формы храняться в списках в столбце `'purpose_lemmas'`. Теперь можно выделить основные цели кредитов.

<a id="category"></a>
### Категоризация данных

Категоризацию будем проводить исходя из целей исследования, чтобы в дальнейшем можно было работать только с необходимыми столбцами:
* создадим отдельную таблицу, в которой будет информация о количестве детей, наличия или отсутствия долга и столбец, в котором будет отображено наличие или отсутствие детей;
* создадим отдельную таблицу с столбцами, отображающими семейное положение и наличие или отсутствие долга;
* создадим таблицу с доходами, дополнительно, разделим доходы на диапазоны значенией и добавим соответствующий столбец;
* создадим таблицу с целями, для этого в столбцах с леммами выделим конкретные теги.

Создадим таблицу с информацией о количестве детей и долге:

In [36]:
# создадим таблицу с информацией и количестве детей и долге
data_children_debt = data.loc[:, ['children', 'debt',]]
data_children_debt.head()

,children,debt
0,1,0
1,1,0
2,0,0
3,3,0
4,0,0


In [37]:
# напишем функцию, которая добавит столбец, отоброжающий наличие или отсутствие детей в семье
def children_status(children):
    if children > 0:
        return 'Дети есть'
    return 'Детей нет'

In [38]:
# применим функцию 'children_status' к таблице с детьми
data_children_debt['children_status'] = data_children_debt['children'].apply(children_status)
data_children_debt.head()

,children,debt,children_status
0,1,0,Дети есть
1,1,0,Дети есть
2,0,0,Детей нет
3,3,0,Дети есть
4,0,0,Детей нет


Создадим таблицу с информацией о семейном положении и наличием или отсутствием долга:

In [39]:
# создадим таблицу c информацией о семейном положении и наличием или отсутствием долга
data_family_debt = data[['family_status_id', 'family_status','debt']]
data_family_debt.head()

,family_status_id,family_status,debt
0,0,женат / замужем,0
1,0,женат / замужем,0
2,0,женат / замужем,0
3,0,женат / замужем,0
4,1,гражданский брак,0


Создадим таблицу, в которую будет входить столбец с задолженностью и столбец с доходами.

In [40]:
data_total_income_debt = data.loc[:,['total_income', 'debt']]
# выведем первые 5 строк получившейся таблицы
data_total_income_debt.head()

,total_income,debt
0,253875,0
1,112080,0
2,145885,0
3,267628,0
4,158616,0


Добавим к этой таблице категории доходов:
* менее 50 тыс.
* 50-100 тыс.
* 100-150 тыс.
* 150-200 тыс.
* 200-250 тыс.
* 250-300 тыс.
* более 300 тыс.

In [41]:
# сделаем функцию, которая присвоит диапозон дохода, в зависимости от его значения
def income_value(total_income):
    if total_income <= 50000:
        return 'менее 50 тыс.'
    if 50000 <= total_income < 100000:
        return '50-100 тыс.'
    if 100000 <= total_income < 150000:
        return '100-150 тыс.'
    if 150000 <= total_income < 200000:
        return '150-200 тыс.'
    if 200000 <= total_income < 250000:
        return '200-250 тыс.'
    if 250000 <= total_income < 300000:
        return '250-300 тыс.'
    return 'более 300 тыс.'

# применим эту функцию к столбцу 'total_income' методом apply и создадим новый столбец 'income_level'
data_total_income_debt['income_value'] = data_total_income_debt['total_income'].apply(income_value)

# выведем первые 5 строк получившейся таблицы
data_total_income_debt.head()


,total_income,debt,income_value
0,253875,0,250-300 тыс.
1,112080,0,100-150 тыс.
2,145885,0,100-150 тыс.
3,267628,0,250-300 тыс.
4,158616,0,150-200 тыс.


Создадим таблицу, в которую будет входить столбец с задолженностью и столбец с целями.

In [42]:
# создадим таблицу, в которую будут входить цели, леммы целей и информация о задолженности
data_purpose_debt = data.loc[:,['purpose', 'purpose_lemmas', 'debt']]
data_purpose_debt['purpose_lemmas'].value_counts()

[автомобиль, \n]                                          972
[свадьба, \n]                                             793
[на,  , проведение,  , свадьба, \n]                       773
[сыграть,  , свадьба, \n]                                 769
[операция,  , с,  , недвижимость, \n]                     675
[покупка,  , коммерческий,  , недвижимость, \n]           662
[операция,  , с,  , жилье, \n]                            652
[покупка,  , жилье,  , для,  , сдача, \n]                 652
[операция,  , с,  , коммерческий,  , недвижимость, \n]    650
[покупка,  , жилье, \n]                                   646
[жилье, \n]                                               646
[покупка,  , жилье,  , для,  , семья, \n]                 638
[строительство,  , собственный,  , недвижимость, \n]      635
[недвижимость, \n]                                        633
[операция,  , со,  , свой,  , недвижимость, \n]           627
[строительство,  , жилой,  , недвижимость, \n]            625
[покупка

Как видно, из полученных в результате лемматизации списков можно выделить несколько основных категорий целей:
* Жилье, недвижмость
* Автомобиль
* Образование
* Свадьба

Преобразуем столбец с леммами целей, заменив строки на выделенные выше категории целей:

In [43]:
# создадим функцию, которая будет получить на вход датафрейм и список основных категорий
# выдаёт 
def purpose_category(purpose):
    if ('жилье' in purpose) or ('недвижимость' in purpose): 
        return 'жилье, недвижимость'
    if 'автомобиль' in purpose: 
        return 'автомобиль'
    if 'образование' in purpose: 
        return 'образование'
    if 'свадьба' in purpose: 
        return 'свадьба'
    return purpose

# заменим значения в столбце со списками лемм на соответсвтующее значение категории цели
data_purpose_debt['purpose_lemmas'] = data_purpose_debt['purpose_lemmas'].apply(purpose_category)
# посчитаем количество уникальных значений теперь в 'purpose_lemmas'
data_purpose_debt['purpose_lemmas'].value_counts()

жилье, недвижимость    10814
автомобиль              4308
образование             4014
свадьба                 2335
Name: purpose_lemmas, dtype: int64

**Вывод**


* Были созданы таблицы с теми столбцами, которые необходимы для проведения исследования;
* категоризированы доходы;
* добавлен столбец описывающий наличие или отсутствие детей;
* были выделены основные категории целей и в соответствии с этим отредактирован столбец с леммами.


<a id="research"></a>
## Исследование заёмщиков

<a id="children_debt"></a>
- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Создадим сводную таблицу, в столбцах количество людей с долгом и без, общее количество клиентов и процент должников:

In [44]:
# создадим сводную таблицу, в которой в качестве индекса будет информация о наличии детей 'children_status'
# в остальных столбцах информация о количестве должников
# в данном случае необходимо посчитать количество методом count(), считать будем по столбцу 'children'
# подошёл бы любой свободный столбец с полным количеством строк
data_children_debt_pivot = data_children_debt.pivot_table(index='children_status', columns='debt', values='children', aggfunc='count')

# добавим столбцец total_clients, в котором будет общее количество клиентов
data_children_debt_pivot['total_clients'] = data_children_debt_pivot[0] + data_children_debt_pivot[1]

# добавим столбцец debt_percent, в котором будет в процентах доля должников
data_children_debt_pivot['debt_percent'] = 100 * data_children_debt_pivot[1] / data_children_debt_pivot['total_clients']

data_children_debt_pivot


debt,0,1,total_clients,debt_percent
children_status,,,,
Детей нет,13044,1063,14107,7.535266
Дети есть,6686,678,7364,9.206953


**Вывод**

* Количество должников среди тех у кого нет детей составляет 7,5%
* Количество должников среди тех у кого есть дети составляет 9,2%

Таким образом, можно сделать вывод, что есть зависимость между наличием детей и возвратом кредита в срок. Те у кого дети есть, чаще являются должниками.

<a id="family_debt"></a>
- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Создадим сводную таблицу, в столбцах количество людей с долгом и без, общее количество клиентов и процент должников:

In [45]:
# создадим сводную таблицу, в которой в качестве индекса будет информация о семейном положении 'family_status'
# в остальных столбцах информация о количестве должников
# в данном случае необходимо посчитать количество методом count(), считать будем по столбцу 'family_status_id'
# подошёл бы любой свободный столбец с полным количеством строк
data_family_debt_pivot = data_family_debt.pivot_table(index='family_status', columns='debt', values='family_status_id', aggfunc='count')

# добавим столбцец total_clients, в котором будет общее количество клиентов
data_family_debt_pivot['total_clients'] = data_family_debt_pivot[0] + data_family_debt_pivot[1]

# добавим столбцец debt_percent, в котором будет в процентах доля должников
data_family_debt_pivot['debt_percent'] = 100 * data_family_debt_pivot[1] / data_family_debt_pivot['total_clients']

data_family_debt_pivot.sort_values(by='debt_percent', ascending=True) # применена сортировка по столбцу с долей должников

debt,0,1,total_clients,debt_percent
family_status,,,,
вдовец / вдова,896,63,959,6.569343
в разводе,1110,85,1195,7.112971
женат / замужем,11413,931,12344,7.542126
гражданский брак,3775,388,4163,9.320202
Не женат / не замужем,2536,274,2810,9.750890


**Вывод**

* У категории вдовец/вдова процент должников 6.5 %
* У категории в разводе процент должников 7.11%
* У категории женат/замужем процент должников 7.5%
* У категории гражданский брак процент должников 9.3%
* У категории не женат/не замужем процент должников 9.75%

Таким образом, можно сделать вывод, что есть зависимость между семейным положением и возвратом кредита.

<a id="income_debt"></a>
- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Создадим сводную таблицу, в столбцах количество людей с долгом и без, общее количество клиентов и процент должников:

In [46]:
# создадим сводную таблицу, в которой в качестве индекса будет информация о диапозоне уровня доходов 'income_value'
# в остальных столбцах информация о количестве должников
# в данном случае необходимо посчитать количество методом count(), считать будем по столбцу 'total_income'
# подошёл бы любой свободный столбец с полным количеством строк
data_total_income_debt_pivot = data_total_income_debt.pivot_table(index='income_value', columns='debt', values='total_income', aggfunc='count')

# добавим столбцец total_clients, в котором будет общее количество клиентов
data_total_income_debt_pivot['total_clients'] = data_total_income_debt_pivot[0] + data_total_income_debt_pivot[1]

# добавим столбцец debt_percent, в котором будет в процентах доля должников
data_total_income_debt_pivot['debt_percent'] = 100 * data_total_income_debt_pivot[1] / data_total_income_debt_pivot['total_clients']

data_total_income_debt_pivot.sort_values(by='debt_percent', ascending=True) # применена сортировка по столбцу с долей должников

debt,0,1,total_clients,debt_percent
income_value,,,,
менее 50 тыс.,349,23,372,6.182796
250-300 тыс.,1242,88,1330,6.616541
более 300 тыс.,1377,106,1483,7.147674
200-250 тыс.,2090,164,2254,7.275954
50-100 тыс.,3760,331,4091,8.090931
150-200 тыс.,4361,405,4766,8.497692
100-150 тыс.,6551,624,7175,8.696864


**Вывод**

* Наибольшее количество должников среди тех, кто находится в диапозоне 50-200 тысяч - более 8%
* Наименьшее количество среди тех, уровень дохода которых менее 50 тыс. Около 6%
* У людей с уровнем дохода более 200 тысяч процент должников 6.6 - 7.3 %

Таким образом, можно сделать вывод, что уровень дохода влияет на возврат кредита.

<a id="purpose_debt"></a>
- Как разные цели кредита влияют на его возврат в срок?

Создадим сводную таблицу, в столбцах количество людей с долгом и без, общее количество клиентов и процент должников:

In [47]:
# создадим сводную таблицу, в которой в качестве индекса будет информация о цели 'purpose_lemmas'
# в остальных столбцах информация о количестве должников
# в данном случае необходимо посчитать количество методом count(), считать будем по столбцу 'purpose'
# подошёл бы любой свободный столбец с полным количеством строк
data_purpose_debt_pivot = data_purpose_debt.pivot_table(index='purpose_lemmas', columns='debt', values='purpose', aggfunc='count')

# добавим столбцец total_clients, в котором будет общее количество клиентов
data_purpose_debt_pivot['total_clients'] = data_purpose_debt_pivot[0] + data_purpose_debt_pivot[1]

# добавим столбцец debt_percent, в котором будет в процентах доля должников
data_purpose_debt_pivot['debt_percent'] = 100 * data_purpose_debt_pivot[1] / data_purpose_debt_pivot['total_clients']

data_purpose_debt_pivot.sort_values(by='debt_percent', ascending=True) # применена сортировка по столбцу с долей должников

debt,0,1,total_clients,debt_percent
purpose_lemmas,,,,
"жилье, недвижимость",10032,782,10814,7.231367
свадьба,2149,186,2335,7.965739
образование,3644,370,4014,9.217738
автомобиль,3905,403,4308,9.354689


**Вывод**

* Наименьшее количество должников среди тех, кто брал кредит на жилье/недижимость - 7.2%
* Среди тех, кто брал кредит на свадьбу количество должников - 8%
* Наибольшее количество должников среди тех, кто брал кредит на образование и автомобиль  - более 9%

Таким образом, можно сделать вывод, что цель кредита влияет на возвраты.

<a id="conclusion"></a>
## Общий вывод

* В данных обнаружены пропуски в столбцах со стажем и ежемесячным доходами. Строки с пропусками совпадают, что говорит о одинаковой природе ошибок. Вероятно, не получилось получить данные от работодателя;
* данные в столбце с ежемесячным доходом были заменены на медианы соответствующего типа занятости;
* в данных со стажем обнаружены и заменены некорректные значения;
* в данных об образовании были обнаружены и устранены многочисленные дупликаты, связанные с регистром;
* была проведена оптимизация типов данных, что уменьшило вес таблицы на 46%;
* выявлены и устранены дупликаты в столбце с информацией о детях;
* была проведена категоризация данных о доходах;
* была проведена категоризация данных о наличии детей;
* была проведена категоризация данных о целях кредита, выделено 4 основных цели: недвижимость, свадьба, образование, автомобиль;
* выявлена зависимость между наличием детей и возвратом кредита в срок: 
    * если дети есть - процент невозврата: 9,2%; 
    * если нет - 7,5%;
* выявлена зависимость между семейным положением и возвратом кредита в срок:
    * меньше всего должников среди вдов/вдовцов: 6%;
    * больше всего среди не женатых/не замужних: 9,2%;
* выявлена зависимость между уровнем дохода и возвратом креди в срок:
    * Наибольшее количество должников среди тех, кто находится в диапозоне 50-200 тысяч - более 8 %;
    * Наименьшее количество среди тех, уровень дохода которых менее 50 тыс. Около 6 %;
    * У людей с уровнем дохода более 200 тысяч процент должников 6.6 - 7.3 %;
* выявлена завсисимость между целью кредита и возвратом кредита в срок
    * Наименьшее количество должников среди тех, кто брал кредит на жилье/недижимость - 7.2%
    * Среди тех, кто брал кредит на свадьбу количество должников - 8%
    * Наибольшее количество должников среди тех, кто брал кредит на образование и автомобиль  - более 9%